In [1]:
%pip install -q ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [6]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import os
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import regex as re
import math
import json
import csv

from stream_analysis.env_ import Env_
from stream_analysis.chat import Chat
from stream_analysis.summary import Summary
from stream_analysis.utils import minutes_to_hhmm, get_secure_dict
%matplotlib inline

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
video_url = 'https://www.youtube.com/live/xnmCKuqoxAM?si=omeWA4mGB7WoSqkP'
chunk_size = 1000

In [3]:
from chat_downloader import ChatDownloader

downloader = ChatDownloader()

In [9]:
import json
env = Env_(video_url)
# chat = downloader.get_chat(env.video_live_url, message_groups=['superchat', 'messages'])
chat = downloader.get_chat(env.video_live_url)
with open('video_data.json', 'w', encoding='utf8') as f:
    json.dump(env.video_data, f, indent=2)
env.video_data


{'title': 'a little surprise!',
 'author': 'Mori Calliope Ch. hololive-EN',
 'author_id': 'UCL_qhgtOy0dy1Agp8vkySQg',
 'original_video_id': 'xnmCKuqoxAM',
 'video_type': 'video',
 'start_time': 1727853376000000.0,
 'end_time': 1727857886000000.0,
 'duration': 4517.0,
 'continuation_info': {'Top chat replay': 'op2w0wRsGl5DaWtxSndvWVZVTk1YM0ZvWjNSUGVUQmtlVEZCWjNBNGRtdDVVMUZuRWd0NGJtMURTM1Z4YjNoQlRSb1Q2cWpkdVFFTkNndDRibTFEUzNWeGIzaEJUU0FCTUFBJTNEQAFyCAgEGAEgACgB',
  'Live chat replay': 'op2w0wRsGl5DaWtxSndvWVZVTk1YM0ZvWjNSUGVUQmtlVEZCWjNBNGRtdDVVMUZuRWd0NGJtMURTM1Z4YjNoQlRSb1Q2cWpkdVFFTkNndDRibTFEUzNWeGIzaEJUU0FCTUFBJTNEQAFyCAgBGAEgACgB'},
 'status': 'past'}

In [59]:
all_keys = set()
chat = downloader.get_chat(env.video_live_url)
for message in chat:
    all_keys.update(message.keys())

all_keys = list(all_keys)

In [7]:
all_keys

['message',
 'message_type',
 'emotes',
 'time_in_seconds',
 'timestamp',
 'author',
 'time_text',
 'action_type',
 'message_id']

In [60]:
emotes = {}
with open('raw.csv', mode='w', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=all_keys)
    
    writer.writeheader()
    
    chat = downloader.get_chat(env.video_live_url)

    for message in chat:
        if message.get('emotes'):
            emotes[message.get('emotes')[0]['name']] = message.get('emotes')
        row = {key: message.get(key, '') for key in all_keys}
        writer.writerow(row)


In [61]:
with open('emotes.json', 'w', encoding='utf8') as f:
    json.dump(emotes, f, indent=2, ensure_ascii=False)

In [72]:
try:
    import sys
    del sys.modules['stream_analysis.types.chat']
except Exception:
    pass

from stream_analysis.types.chat import ChatItem

df_raw = pd.DataFrame()

chat = downloader.get_chat(env.video_live_url)
for chat_item in chat:
    chat_item = ChatItem(chat_item, env)
    df = pd.concat([df, pd.DataFrame([chat_item])], ignore_index=True)
df_raw

In [13]:
df = pd.read_csv('raw.csv')
df.head()

,message,message_type,emotes,time_in_seconds,timestamp,author,time_text,action_type,message_id
0,:_deadbeats::_deadbeats::_deadbeats::_deadbeats:,text_message,[{'id': 'UCL_qhgtOy0dy1Agp8vkySQg/lr5OYciWAcG3...,-20.0,1727853343217886,"{'name': 'Flugmorph', 'images': [{'url': 'http...",-0:20,add_chat_item,ChwKGkNJR1FfS1dUNzRnREZSMEkxZ0FkX0pZOFdR
1,lmao,text_message,NaN,-19.0,1727853344811803,"{'name': 'Alkanshel', 'images': [{'url': 'http...",-0:19,add_chat_item,ChwKGkNMckIzYWFUNzRnREZkd3VyUVlkQm1nS253
2,ikzzzzzzzzzzzzz,text_message,NaN,-17.0,1727853346449862,"{'name': 'steren steven', 'images': [{'url': '...",-0:17,add_chat_item,ChwKGkNPeWN3YWVUNzRnREZWZ3FyUVlkOFFnd3NB
3,:face-red-heart-shape:,text_message,[{'id': 'UCkszU2WH9gy1mb0dV-11UJg/m8MfY4jbFsWJ...,-17.0,1727853346474954,"{'name': 'Harmony', 'images': [{'url': 'https:...",-0:17,add_chat_item,ChwKGkNOQ0l3NmVUNzRnREZXb3YxZ0FkUEQwZDJB
4,Thanks Biv,text_message,NaN,-16.0,1727853347241507,"{'name': 'Rave', 'images': [{'url': 'https://y...",-0:16,add_chat_item,ChwKGkNPdnI4YWVUNzRnREZVUFFsQWtkQVBFTEJR


In [68]:
try:
    import sys
    del sys.modules['stream_analysis.types.chat']
except Exception:
    pass

from stream_analysis.analyzer import get_sentiment
from stream_analysis.types.chat import StrReplaced

message = StrReplaced(df['message'].iloc[7])
display(message)
get_sentiment(message)

':eyes::eyes:'

(0.7052189707756042, 0.1158713772892952, 0.17890962958335876, 'neutral')